### Summarizing Large Documents using `map_reduce`

In [1]:
from langchain.chat_models import ChatOpenAI
from langchain import PromptTemplate
from langchain.chains import summarize
from langchain.text_splitter import RecursiveCharacterTextSplitter
from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv(), override=True)

True

In [2]:
with open('../2 QA on Private Documents/files/sj.txt', encoding='utf-8') as f:
    text = f.read()

llm = ChatOpenAI(temperature=0, model_name='gpt-3.5-turbo')
llm.get_num_tokens(text)

2653

In [3]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=10000, chunk_overlap=50)
chunks = text_splitter.create_documents([text])
len(chunks)

2

In [4]:
chain = summarize.load_summarize_chain(
    llm,
    chain_type='map_reduce',
    verbose=True
)
output_summary = chain.run(chunks)
print(output_summary)



> Entering new MapReduceDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Write a concise summary of the following:


"I am honored to be with you today at your commencement from one of the finest universities in the world. I never graduated from college. Truth be told, this is the closest I’ve ever gotten to a college graduation. Today I want to tell you three stories from my life. That’s it. No big deal. Just three stories.

The first story is about connecting the dots.

I dropped out of Reed College after the first 6 months, but then stayed around as a drop-in for another 18 months or so before I really quit. So why did I drop out?

It started before I was born. My biological mother was a young, unwed college graduate student, and she decided to put me up for adoption. She felt very strongly that I should be adopted by college graduates, so everything was all set for me to be adopted at birth by a lawyer and his wife. Except that when I popped out


> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:
Write a concise summary of the following:


"In this speech, Steve Jobs shares three stories from his life. The first story is about dropping out of college and how it led him to take a calligraphy class, which later influenced the design of the Macintosh computer. The second story is about getting fired from Apple and how it ultimately led him to start new ventures and find love. The third story is about his experience with cancer and how it reminded him of the importance of living each day as if it were his last. Jobs emphasizes the importance of following one's passion and not settling for anything less.

The speaker reflects on the inevitability of death and its role in making way for new life. They encourage the audience to live their own lives and not be influenced by others, urging them to follow their hearts and intuition. The speaker also mentions a publication called The Whole Earth Catalog and its

In [5]:
chain.llm_chain.prompt.template

'Write a concise summary of the following:\n\n\n"{text}"\n\n\nCONCISE SUMMARY:'

In [6]:
chain.combine_document_chain.llm_chain.prompt.template

'Write a concise summary of the following:\n\n\n"{text}"\n\n\nCONCISE SUMMARY:'

### `map_reduce` with Custom Prompts

In [7]:
map_prompt = """
Write a short and concise summary of the following:
Text: `{text}`
CONCISE SUMMARY:
"""
map_prompt_template = PromptTemplate(
    input_variables=['text'],
    template=map_prompt
)


combine_prompt = """
Write a concise summary of the following text that covers the key points.
Add a title to the summary.
Start your summary with an INTRODUCTION PARAGRAPH that gives an overview of the topic FOLLOWED
by BULLET POINTS if possible AND end the summary with a CONCLUSION PHRASE.
Text: `{text}`
"""
combine_prompt_template = PromptTemplate(
    template=combine_prompt, 
    input_variables=['text']
)


summary_chain = summarize.load_summarize_chain(
    llm=llm,
    chain_type='map_reduce',
    map_prompt=map_prompt_template,
    combine_prompt=combine_prompt_template,
    verbose=True
)
output = summary_chain.run(chunks)



> Entering new MapReduceDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Write a short and concise summary of the following:
Text: `I am honored to be with you today at your commencement from one of the finest universities in the world. I never graduated from college. Truth be told, this is the closest I’ve ever gotten to a college graduation. Today I want to tell you three stories from my life. That’s it. No big deal. Just three stories.

The first story is about connecting the dots.

I dropped out of Reed College after the first 6 months, but then stayed around as a drop-in for another 18 months or so before I really quit. So why did I drop out?

It started before I was born. My biological mother was a young, unwed college graduate student, and she decided to put me up for adoption. She felt very strongly that I should be adopted by college graduates, so everything was all set for me to be adopted at birth by a lawyer and his wife. Except that wh


> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:

Write a concise summary of the following text that covers the key points.
Add a title to the summary.
Start your summary with an INTRODUCTION PARAGRAPH that gives an overview of the topic FOLLOWED
by BULLET POINTS if possible AND end the summary with a CONCLUSION PHRASE.
Text: `In this speech, Steve Jobs shares three stories from his life. The first story is about dropping out of college and how it led him to take a calligraphy class, which later influenced the design of the Macintosh computer. The second story is about getting fired from Apple and how it ultimately led him to start new ventures and find success. The third story is about his experience with cancer and how it reminded him of the importance of living each day to the fullest. Jobs encourages the audience to follow their passions and not settle for anything less.

The text discusses the inevitability of death and how it is a necessary part of l

In [8]:
print(output)

Title: Steve Jobs' Speech on Life Lessons and Following Passion

Introduction:
In this speech, Steve Jobs shares three stories from his life, highlighting the importance of following one's passion and not settling for anything less. He discusses how his experiences with dropping out of college, getting fired from Apple, and battling cancer shaped his perspective on life.

Key Points:
- Story 1: Jobs dropped out of college and took a calligraphy class, which later influenced the design of the Macintosh computer.
- Story 2: Getting fired from Apple led Jobs to start new ventures and find success.
- Story 3: Jobs' experience with cancer reminded him of the importance of living each day to the fullest.
- Jobs encourages the audience to follow their passions and not be influenced by others' opinions.
- The text discusses the inevitability of death and emphasizes the importance of living one's own life.
- Following one's heart and intuition is emphasized.
- The text mentions The Whole Earth 